In [ ]:
import $ivy.`org.locationtech.geotrellis::geotrellis-raster:3.5.0`
import $ivy.`org.slf4j:slf4j-simple:1.7.30`

In [ ]:
import geotrellis.vector._
import geotrellis.raster._
import geotrellis.raster.geotiff.GeoTiffRasterSource

In [ ]:
val uri = "https://geotrellis-demo.s3.amazonaws.com/cogs/harrisburg-pa/elevation.tif"
val rs = GeoTiffRasterSource(uri)

In [ ]:
val raster = rs.read().get

In [ ]:
import geotrellis.proj4.LatLng

// Carlisle, PA
 val geom = Polygon(
   LineString(
     List(
       (-77.25414276123047, 40.168642443446366),
       (-77.1188735961914, 40.168642443446366),
       (-77.1188735961914, 40.24939807617368),
       (-77.25414276123047, 40.24939807617368),
       (-77.25414276123047, 40.168642443446366)
     )
   )
 )
 val geomInRsCrs = geom.reproject(LatLng, rs.crs)

In [ ]:
val raster0 = Raster(raster.tile.band(0), rs.extent)

In [ ]:
import geotrellis.raster.summary.polygonal._
import geotrellis.raster.summary.polygonal.visitors.FastMapHistogramVisitor

val result = raster0.polygonalSummary(geomInRsCrs, FastMapHistogramVisitor)

In [ ]:
val histogram = result.toOption.get
histogram.statistics